In [1]:
# ==============================================================================
#           KODE #1: PEMBUATAN FILE MODEL
# ==============================================================================
import pandas as pd
import numpy as np
import re
import joblib
from google.colab import files
from google.colab import drive

def hitung_dan_tambahkan_skor_gizi(df):
    print("Menghitung skor gizi untuk setiap produk...")
    BOBOT_PROTEIN, BOBOT_GULA, BOBOT_LEMAK_JENUH, BOBOT_SODIUM, BOBOT_SERAT = 2.0, -1.5, -2.5, -1.0, 1.5
    skor_mentah = pd.Series(0, index=df.index)
    if 'protein' in df.columns: skor_mentah += df['protein'] * BOBOT_PROTEIN
    if 'sugar' in df.columns: skor_mentah += df['sugar'] * BOBOT_GULA
    if 'saturated_fat' in df.columns: skor_mentah += df['saturated_fat'] * BOBOT_LEMAK_JENUH
    if 'sodium_mg' in df.columns: skor_mentah += (df['sodium_mg'] / 1000) * BOBOT_SODIUM
    if 'fiber' in df.columns: skor_mentah += df['fiber'] * BOBOT_SERAT
    min_skor, max_skor = skor_mentah.min(), skor_mentah.max()
    df['skor_gizi'] = 50 if max_skor == min_skor else (1 + 99 * (skor_mentah - min_skor) / (max_skor - min_skor))
    df['skor_gizi'] = df['skor_gizi'].round(1)
    print("Skor gizi berhasil dihitung.")
    return df

print("Menghubungkan ke Google Drive...")
drive.mount('/content/drive', force_remount=True)

print("\nMemuat dataset mentah...")
try:
    df = pd.read_csv('indonesia_products (4).csv')
except FileNotFoundError:
    print("File tidak ditemukan. Silakan unggah 'indonesia_products (4).csv'")
    uploaded = files.upload()
    df = pd.read_csv(next(iter(uploaded)))

df.dropna(subset=['product_name'], inplace=True)

COLUMN_MAPPING = {
    'product_name': 'product_name', 'brand': 'brands', 'net_weight': 'net_weight_g',
    'energy_kj': 'energy_100g', 'fat': 'fat_100g', 'protein': 'proteins_100g',
    'carbs': 'carbohydrates_100g', 'salt': 'salt_100g', 'sugar': 'sugars_100g',
    'saturated_fat': 'saturated-fat_100g', 'ingredients': 'ingredients_text',
    'fiber': 'fiber_100g'
}

print("Memproses data...")
existing_cols = [col for col in COLUMN_MAPPING.values() if col in df.columns]
df_filtered = df[existing_cols].copy()
df_filtered.rename(columns={v: k for k, v in COLUMN_MAPPING.items()}, inplace=True)

if 'energy_kj' in df_filtered.columns: df_filtered['energy_kal'] = df_filtered['energy_kj'] / 4.184
if 'salt' in df_filtered.columns: df_filtered['sodium_mg'] = (df_filtered['salt'] / 2.5) * 1000
df_filtered.dropna(subset=['net_weight', 'energy_kal'], inplace=True)

nutrients_to_fill = ['fat', 'protein', 'carbs', 'sodium_mg', 'sugar', 'saturated_fat', 'ingredients', 'fiber']
for col in nutrients_to_fill:
    if col in df_filtered.columns:
        if df_filtered[col].dtype == 'object': df_filtered[col] = df_filtered[col].fillna('Tidak ada informasi')
        else: df_filtered[col] = df_filtered[col].fillna(0)

df_with_score = hitung_dan_tambahkan_skor_gizi(df_filtered)

nutrients_to_process = ['energy_kal', 'fat', 'protein', 'carbs', 'sodium_mg', 'sugar', 'fiber']
for col in nutrients_to_process:
    if col in df_with_score.columns: df_with_score[f'total_{col}'] = (df_with_score[col] / 100) * df_with_score['net_weight']

df_processed = df_with_score.rename(columns={
    'net_weight':'nett_weight_numeric', 'total_fat': 'total_fat_g',
    'total_protein': 'total_protein_g', 'total_carbs': 'total_carbohydrate_g',
    'total_sugar': 'total_sugar_g', 'total_fiber': 'total_fiber_g'
})

final_columns = [
    'product_name', 'brand', 'nett_weight_numeric', 'skor_gizi', 'ingredients',
    'total_energy_kal', 'total_fat_g', 'total_protein_g',
    'total_carbohydrate_g', 'total_sodium_mg', 'total_sugar_g',
    'total_fiber_g'
]
df_final = df_processed[[col for col in final_columns if col in df_processed.columns]]

MODEL_SAVE_PATH = '/content/drive/MyDrive/rekomendasi_gizi.joblib'
joblib.dump(df_final, MODEL_SAVE_PATH)

print("\n==============================================")
print("✅ PEMBUATAN FILE MODEL (DENGAN SERAT) SELESAI.")
print(f"File model telah disimpan di: {MODEL_SAVE_PATH}")
print("==============================================")

Menghubungkan ke Google Drive...
Mounted at /content/drive

Memuat dataset mentah...
Memproses data...
Menghitung skor gizi untuk setiap produk...
Skor gizi berhasil dihitung.

✅ PEMBUATAN FILE MODEL (DENGAN SERAT) SELESAI.
File model telah disimpan di: /content/drive/MyDrive/rekomendasi_gizi.joblib


In [2]:
%%writefile logika_rekomendasi.py
import pandas as pd
import random
import numpy as np


DATA_AKG_PMK_2019 = [
    (10, 12, 'semua', {'energy_kal': 2000, 'protein_g': 50, 'fat_g': 65, 'carbohydrate_g': 300, 'fiber_g': 28}),
    (13, 15, 'laki-laki', {'energy_kal': 2400, 'protein_g': 65, 'fat_g': 80, 'carbohydrate_g': 350, 'fiber_g': 34}),
    (13, 15, 'perempuan', {'energy_kal': 2050, 'protein_g': 65, 'fat_g': 70, 'carbohydrate_g': 300, 'fiber_g': 29}),
    (16, 18, 'laki-laki', {'energy_kal': 2650, 'protein_g': 75, 'fat_g': 85, 'carbohydrate_g': 400, 'fiber_g': 37}),
    (16, 18, 'perempuan', {'energy_kal': 2100, 'protein_g': 65, 'fat_g': 70, 'carbohydrate_g': 300, 'fiber_g': 29}),
    (19, 29, 'laki-laki', {'energy_kal': 2650, 'protein_g': 65, 'fat_g': 75, 'carbohydrate_g': 430, 'fiber_g': 38}),
    (19, 29, 'perempuan', {'energy_kal': 2250, 'protein_g': 60, 'fat_g': 65, 'carbohydrate_g': 360, 'fiber_g': 32}),
    (30, 49, 'laki-laki', {'energy_kal': 2550, 'protein_g': 65, 'fat_g': 70, 'carbohydrate_g': 415, 'fiber_g': 36}),
    (30, 49, 'perempuan', {'energy_kal': 2150, 'protein_g': 60, 'fat_g': 60, 'carbohydrate_g': 340, 'fiber_g': 30}),
    (50, 64, 'laki-laki', {'energy_kal': 2150, 'protein_g': 65, 'fat_g': 60, 'carbohydrate_g': 340, 'fiber_g': 30}),
    (50, 64, 'perempuan', {'energy_kal': 1800, 'protein_g': 60, 'fat_g': 50, 'carbohydrate_g': 280, 'fiber_g': 25}),
    (65, 80, 'laki-laki', {'energy_kal': 1800, 'protein_g': 64, 'fat_g': 50, 'carbohydrate_g': 275, 'fiber_g': 25}),
    (65, 80, 'perempuan', {'energy_kal': 1550, 'protein_g': 58, 'fat_g': 45, 'carbohydrate_g': 235, 'fiber_g': 22}),
    (80, 999, 'laki-laki', {'energy_kal': 1600, 'protein_g': 64, 'fat_g': 45, 'carbohydrate_g': 235, 'fiber_g': 22}),
    (80, 999, 'perempuan', {'energy_kal': 1400, 'protein_g': 58, 'fat_g': 40, 'carbohydrate_g': 205, 'fiber_g': 20})
]
BATAS_GULA_HARIAN = [ (0, 1, 0), (2, 4, 16), (4, 7, 20), (7, 10, 23), (10, 13, 27), (13, 15, 32), (18, 150, 50) ]

def get_full_day_targets(umur, jenis_kelamin):
    jk = jenis_kelamin.lower()
    for min_age, max_age, gender, targets in DATA_AKG_PMK_2019:
        if gender == 'semua' or jk.startswith(gender[0]):
            if umur >= min_age and umur < max_age + 1:
                final_targets = targets.copy()
                final_targets['sodium_mg_limit'] = 1500.0
                batas_gula = 50.0
                for min_a, max_a, limit in BATAS_GULA_HARIAN:
                    if umur >= min_a and umur <= max_a:
                        batas_gula = float(limit)
                        break
                final_targets['sugar_g_limit'] = batas_gula
                return final_targets
    return None

def calculate_remaining_needs(full_targets, consumed):
    remaining = {}
    for nutrient_key, target_value in full_targets.items():
        consumed_key = nutrient_key.replace('_limit', '')
        consumed_value = consumed.get(consumed_key, 0)
        remaining[nutrient_key] = max(0, target_value - consumed_value)
    return remaining

def generate_recommendations(df, needs, num_prod=3, iters=30000):
    best_combo, best_score = None, float('inf')
    sisa_gula_limit, sisa_sodium_limit = needs.get('sugar_g_limit', 0), needs.get('sodium_mg_limit', 0)
    for _ in range(iters):
        combo = df.sample(n=num_prod) if len(df) >= num_prod else df
        rekom_gula, rekom_sodium = combo['total_sugar_g'].sum(), combo['total_sodium_mg'].sum()
        if rekom_gula > sisa_gula_limit or rekom_sodium > sisa_sodium_limit:
            continue

        nutr = {'energy_kal': combo['total_energy_kal'].sum(), 'protein_g': combo['total_protein_g'].sum(),
                'fat_g': combo['total_fat_g'].sum(), 'carbohydrate_g': combo['total_carbohydrate_g'].sum(),
                'fiber_g': combo['total_fiber_g'].sum()}

        w_kal, w_makro, w_batas = 0.5, 0.4, 0.1
        score_kal = abs(nutr['energy_kal'] - needs['energy_kal']) / needs['energy_kal'] if needs['energy_kal'] > 0 else 0


        score_pro = abs(nutr['protein_g'] - needs['protein_g']) / needs['protein_g'] if needs['protein_g'] > 0 else 0
        score_fat = abs(nutr['fat_g'] - needs['fat_g']) / needs['fat_g'] if needs['fat_g'] > 0 else 0
        score_car = abs(nutr['carbohydrate_g'] - needs['carbohydrate_g']) / needs['carbohydrate_g'] if needs['carbohydrate_g'] > 0 else 0
        score_fib = abs(nutr['fiber_g'] - needs['fiber_g']) / needs['fiber_g'] if needs.get('fiber_g', 0) > 0 else 0
        score_makro_avg = (score_pro + score_fat + score_car + score_fib) / 4

        score_gula = (rekom_gula / sisa_gula_limit) if sisa_gula_limit > 0 else (1 if rekom_gula > 0 else 0)
        score_sod = (rekom_sodium / sisa_sodium_limit) if sisa_sodium_limit > 0 else (1 if rekom_sodium > 0 else 0)
        score_limit = (score_gula + score_sod) / 2

        total_score = (score_kal * w_kal) + (score_makro_avg * w_makro) + (score_limit * w_batas)

        if total_score < best_score:
            best_score, best_combo = total_score, combo

    return best_combo

print("File 'logika_rekomendasi.py' (dengan logika serat) berhasil dibuat.")

Writing logika_rekomendasi.py


In [5]:
# ==============================================================================
#           KODE INFERENSI
# ==============================================================================
import pandas as pd
import joblib
from google.colab import drive
import sys

pd.options.display.float_format = '{:,.2f}'.format
sys.path.append('/content/')

try:
    from logika_rekomendasi import get_full_day_targets, calculate_remaining_needs, generate_recommendations
    print("✅ File Logika (logika_rekomendasi.py) berhasil diimpor.")
except ImportError:
    print("❌ ERROR: File 'logika_rekomendasi.py' tidak ditemukan.")
    exit()

try:
    drive.mount('/content/drive', force_remount=True)
    MODEL_PATH = '/content/drive/MyDrive/rekomendasi_gizi.joblib'
    df_model = joblib.load(MODEL_PATH)
    print(f"✅ File Dataset ({MODEL_PATH}) berhasil dimuat.")
except FileNotFoundError:
    print(f"❌ ERROR: File data tidak ditemukan. Jalankan skrip pembuat model terlebih dahulu.")
    exit()

try:
    print("\n--- SILAKAN MASUKKAN PROFIL ANDA ---")
    umur = int(input("Masukkan umur Anda (contoh: 30): "))
    jenis_kelamin = input("Masukkan jenis kelamin (Laki-laki / Perempuan): ")

    print("\n--- MASUKKAN TOTAL NUTRISI YANG SUDAH DIKONSUMSI HARI INI ---")
    konsumsi = {
        'energy_kal': float(input("Kalori (kkal): ")), 'protein_g': float(input("Protein (g): ")),
        'fat_g': float(input("Lemak (g): ")), 'carbohydrate_g': float(input("Karbohidrat (g): ")),
        'sodium_mg': float(input("Sodium (mg): ")), 'sugar_g': float(input("Gula (g): ")),
        'fiber_g': float(input("Serat (g): "))
    }

    target_harian = get_full_day_targets(umur, jenis_kelamin)
    if target_harian is None:
        print("Profil tidak ditemukan dalam data AKG.")
        exit()

    print("\n--- ANALISIS KONSUMSI AWAL ---")
    map_nama = {'energy_kal': 'Kalori', 'protein_g': 'Protein', 'fat_g': 'Lemak',
                'carbohydrate_g': 'Karbohidrat', 'sugar_g': 'Gula', 'sodium_mg': 'Sodium',
                'fiber_g': 'Serat'}
    pesan_analisis = []
    for key, target_value in target_harian.items():
        consumed_key = key.replace('_limit', '')
        consumed_value = konsumsi.get(consumed_key, 0)
        nama_nutrisi = map_nama.get(consumed_key, '').capitalize()
        if not nama_nutrisi: continue

        if '_limit' in key:
            if consumed_value > target_value: print(f"⚠️  {nama_nutrisi} ({consumed_value:,.0f}) MELEBIHI batas harian ({target_value:,.0f}).")
            else: print(f"✅ Konsumsi {nama_nutrisi} masih di bawah batas aman.")
        else:
            if consumed_value >= target_value * 0.95: print(f"✅ Target {nama_nutrisi} ({consumed_value:,.0f}) sudah terpenuhi.")

    print("\nMemproses rekomendasi untuk Anda...")
    sisa_kebutuhan = calculate_remaining_needs(target_harian, konsumsi)
    rekomendasi = generate_recommendations(df_model, sisa_kebutuhan, num_prod=3)

    print("\n================== HASIL REKOMENDASI ==================")
    if rekomendasi is not None:
        defisiensi = {
            'Protein': sisa_kebutuhan.get('protein_g', 0) / target_harian.get('protein_g', 1),
            'Lemak': sisa_kebutuhan.get('fat_g', 0) / target_harian.get('fat_g', 1),
            'Karbohidrat': sisa_kebutuhan.get('carbohydrate_g', 0) / target_harian.get('carbohydrate_g', 1),
            'Serat': sisa_kebutuhan.get('fiber_g', 0) / target_harian.get('fiber_g', 1)
        }
        nutrisi_utama_kurang = sorted([nama for nama, skor in defisiensi.items() if skor > 0.1], key=lambda x: defisiensi[x], reverse=True)

        if not nutrisi_utama_kurang:
            judul_rekomendasi = "Berikut Rekomendasi Makanan Pilihan Penambah Energi"
        else:
            judul_rekomendasi = f"Berikut Rekomendasi Makanan Tinggi {' dan '.join(nutrisi_utama_kurang)}"

        print(judul_rekomendasi)
        print("-" * (len(judul_rekomendasi) + 2))


        for index, row in rekomendasi.iterrows():
            print(f"\n-> {row['product_name']} (Skor Gizi: {row['skor_gizi']})")
            print(f"   - Kalori     : {row.get('total_energy_kal', 0):.1f} kkal")
            print(f"   - Protein    : {row.get('total_protein_g', 0):.1f} g")
            print(f"   - Lemak      : {row.get('total_fat_g', 0):.1f} g")
            print(f"   - Karbohidrat: {row.get('total_carbohydrate_g', 0):.1f} g")
            print(f"   - Serat      : {row.get('total_fiber_g', 0):.1f} g")
            print(f"   - Gula       : {row.get('total_sugar_g', 0):.1f} g")
            print(f"   - Sodium     : {row.get('total_sodium_mg', 0):.0f} mg")

    else:
        print("Tidak dapat menemukan rekomendasi yang cocok.")
    print("\n=====================================================")

except Exception as e:
    print(f"\n❌ Terjadi kesalahan: {e}")

✅ File Logika (logika_rekomendasi.py) berhasil diimpor.
Mounted at /content/drive
✅ File Dataset (/content/drive/MyDrive/rekomendasi_gizi.joblib) berhasil dimuat.

--- SILAKAN MASUKKAN PROFIL ANDA ---
Masukkan umur Anda (contoh: 30): 25
Masukkan jenis kelamin (Laki-laki / Perempuan): Laki-laki

--- MASUKKAN TOTAL NUTRISI YANG SUDAH DIKONSUMSI HARI INI ---
Kalori (kkal): 1500
Protein (g): 20
Lemak (g): 10
Karbohidrat (g): 240
Sodium (mg): 550
Gula (g): 20
Serat (g): 10

--- ANALISIS KONSUMSI AWAL ---
✅ Konsumsi Sodium masih di bawah batas aman.
✅ Konsumsi Gula masih di bawah batas aman.

Memproses rekomendasi untuk Anda...

================== HASIL REKOMENDASI ==================
Berikut Rekomendasi Makanan Tinggi Lemak dan Serat dan Protein dan Karbohidrat
--------------------------------------------------------------------------------

-> Nutty Chocolate Cookies (Skor Gizi: 79.1)
   - Kalori     : 999.0 kkal
   - Protein    : 10.0 g
   - Lemak      : 50.0 g
   - Karbohidrat: 120.0 g
  